In [1]:
%store -z # remove all variables from variable storage

In [2]:
# load credentials from environment variables
%load_ext dotenv
%dotenv

# util
import numpy as np
import boto3
import json
import csv

# date & time
import time
from datetime import timezone, date, datetime
from dateutil.relativedelta import relativedelta as rdelta
from dateutil.rrule import rrule, MONTHLY

# Oauth
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

## Get authorization token

In [3]:
# Your client credentials
client_id = %env SH_CLIENT_ID
client_secret = %env SH_CLIENT_SECRET

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

## Configure request (evalscript)

Enter start and end date, input bands, indices. The resulting files will have two time intervals per month, being split at `day_of_new_interval`.

In [4]:
startdate = date(2017,9,1) # Y,M,D
enddate = date(2018,11,30)  # Y,M,D

input_bands = [
    "B02",
    "B03",
    "B04",
    "B05",
    "B06",
    "B07",
    "B08",
    "B8A",
    "B11",
    "B12"
]


indices = [
    "NDVI",
    "GNDVI",
    "BNDVI",
    "CVI",
    "NDSI",
    "NDWI"
]

bucket_name = "eox-masterdatacube"

day_of_new_interval = 16 # leave this unchanged in most of the cases

### Calculate parameters

In [5]:
starttime = datetime(*startdate.timetuple()[:6])
endtime = datetime(*enddate.timetuple()[:6])

d=day_of_new_interval
dates = list(rrule(MONTHLY, dtstart=startdate, until=enddate, bymonthday=[1,d-1,d,-1]))
dates = [starttime] + dates if dates[0] != starttime else dates
dates = dates + [endtime] if dates[-1] != endtime else dates

starts = dates[0::2]
starts = [int(d.timestamp()) for d in starts] # timestamps for arithmetic
ends   = [d+rdelta(hour=23, minute=59, second=59) for d in dates[1::2]]
ends   = [int(d.timestamp()) for d in ends]   # timestamps for arithmetic
avg_times = list(np.mean(list(zip(starts,ends)), axis=1))
avg_times = [datetime.utcfromtimestamp(a) for a in avg_times]
avg_times = [dt.isoformat() for dt in avg_times]

In [6]:
masks = ["SCL", "dataMask"] # SCL ... Scene Classification Layer

output_bands = input_bands + indices
output_array =  [ { 'id': "\"" + ob + "\"", 'bands': len(avg_times), "sampleType": "SampleType.UINT16"} for ob in output_bands ]
for oa in output_array:
    if oa["id"] == '"CVI"':
        oa["sampleType"] = "SampleType.FLOAT32"
output_array = str(output_array).replace("'", '')

int_bands = '{' + ','.join([f'{ib}: []' for ib in input_bands]) + '}'
results_object = '{' + ','.join([f'{ob}: []' for ob in output_bands]) + '}'
debug_results = '{' + ','.join([f"{output_bands[i]}: [{i+1}]" for i in range(len(output_bands))]) + '}'
responses = [{"identifier": ob,"format": {"type": "image/tiff"}} for ob in output_bands]

### Evalscript & Payload

In [7]:
with open('evalscript.js', 'r') as file:
    evalscript = file.read()
    
evalscript = evalscript.format(
    bands              =str(input_bands+masks),
    output_array       =output_array,
    results_object     =results_object,
    day_of_new_interval=day_of_new_interval,
    enddate_unix       =datetime(*enddate.timetuple()[:3],23,59,59).timestamp()*1000,
    debug_results      =debug_results,
    int_bands          =int_bands,
    indices            =indices,
    avg_times          =avg_times
)

In [8]:
payload = {
  "processRequest": {
    "input": {
      "bounds": {
        "properties": {
          "crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
        },
        "bbox": [16.2685837616047984,47.6328390028652180,16.5522285726248839,47.8232776963671000]
      },
      "data": [
        {
          "location": "AWS:eu-central-1",
          "type": "S2L2A",
          "dataFilter": {
            "timeRange": {
              "from": starttime.isoformat() + 'Z',
              "to": endtime.isoformat() + 'Z'
            },
            "mosaickingOrder": "mostRecent",
            "maxCloudCoverage": 100,
            "previewMode": "DETAIL"
          }
        }
      ]
    },
    "output": {
      "responses": [*responses]
    },
    "evalscript": evalscript
  },
  "tilingGridId": 0,
  "bucketName": bucket_name,
  "resolution": 10.0,
  "description": f"Test Vorarlberg 16 {starttime.strftime('%Y%m%d')}_{endtime.strftime('%Y%m%d')}"
}

headers = {
  #'Accept': 'application/tar'
}

## Send request

In [9]:
def generate_url(request_id="", action=""):
    url = 'https://services.sentinel-hub.com/batch/v1/process/'
    if request_id:
        url += f'{request_id}/'
        if action:
            url += f'{action}'
    return url

In [10]:
start_time = time.time()

response = oauth.request("POST", generate_url(), headers=headers, json = payload).json()
request_id = response["id"]
vE = response["valueEstimate"]
print(f"""Status of request {request_id}: {response['status']}
Value Estimate: {vE:.3f}""")

oauth.request("POST", generate_url(request_id, 'start'))
print('Processing started.')

s3 = boto3.resource('s3')
bk = s3.Bucket(bucket_name)

bk.put_object(Key=request_id + '/userdata.json', Body=json.dumps({
    'bands': output_bands,
    'request_id': request_id,
#     'tiles': tiles,
    'time': avg_times
}))
print('Metadata saved to bucket')

Status of request 90442256-625e-45de-9838-910e201617c7: CREATED
Value Estimate: 28423.307
Processing started.
Metadata saved to bucket


In [11]:
%store bucket_name
%store request_id
%store

Stored 'bucket_name' (str)
Stored 'request_id' (str)
Stored variables and their in-db values:
bucket_name             -> 'eox-masterdatacube'
request_id              -> '90442256-625e-45de-9838-910e201617c7'
